# Preparing Dataset

## Reading dataset

In [236]:
import pandas as pd
pd.options.mode.chained_assignment = None
all_stock_df = pd.read_csv("D:/Documents/College docs/Sem III/Mini project/data/NIFTY50_all_modified.csv")
all_stock_df["Date"] = pd.to_datetime(all_stock_df["Date"])
finance_minister_df = pd.read_csv(
    'D:/Documents/College docs/Sem III/Mini project/data/finance_minister_data.csv')
finance_minister_df["Date from"] = pd.to_datetime(
    finance_minister_df["Date from"])
finance_minister_df["Date to"] = pd.to_datetime(finance_minister_df["Date to"])

## Making a dictionary with industry as key and it's stocks as values

In [237]:
industries = {}
for industry_name in list(all_stock_df.Industry.unique()):
    industries[industry_name] = list(
        all_stock_df.loc[all_stock_df.Industry == industry_name, "Symbol"].unique())
industries

{'SERVICES': ['MUNDRAPORT', 'ADANIPORTS'],
 'CONSUMER GOODS': ['ASIANPAINT',
  'BRITANNIA',
  'HINDLEVER',
  'HINDUNILVR',
  'ITC',
  'NESTLEIND',
  'TITAN'],
 'FINANCIAL SERVICES': ['UTIBANK',
  'AXISBANK',
  'BAJAJFINSV',
  'BAJAUTOFIN',
  'BAJFINANCE',
  'HDFC',
  'HDFCBANK',
  'ICICIBANK',
  'INDUSINDBK',
  'KOTAKMAH',
  'KOTAKBANK',
  'SBIN'],
 'AUTOMOBILE': ['BAJAJ-AUTO',
  'EICHERMOT',
  'HEROHONDA',
  'HEROMOTOCO',
  'M&M',
  'MARUTI',
  'TELCO',
  'TATAMOTORS'],
 'TELECOM': ['BHARTI', 'BHARTIARTL'],
 'ENERGY': ['BPCL', 'GAIL', 'IOC', 'NTPC', 'ONGC', 'POWERGRID', 'RELIANCE'],
 'PHARMA': ['CIPLA', 'DRREDDY', 'SUNPHARMA'],
 'METALS': ['COALINDIA',
  'HINDALC0',
  'HINDALCO',
  'JSWSTL',
  'JSWSTEEL',
  'TISCO',
  'TATASTEEL',
  'SESAGOA',
  'SSLT',
  'VEDL'],
 'CEMENT & CEMENT PRODUCTS': ['GRASIM', 'SHREECEM', 'ULTRACEMCO'],
 'IT': ['HCLTECH', 'INFOSYSTCH', 'INFY', 'TCS', 'TECHM', 'WIPRO'],
 'CONSTRUCTION': ['LT'],
 'FERTILISERS & PESTICIDES': ['UNIPHOS', 'UPL'],
 'MEDIA & ENTERT

# Graphs


## Stocks in a Sector

In [238]:
import plotly.express as px
industry = list(industries.keys())[0]
roll_data = True, 50
stock_df = all_stock_df.loc[all_stock_df["Industry"] == industry]
stock_df["Rolling mean"] = stock_df["Close"].rolling(
    window=roll_data[1]).mean()
fig = px.line(stock_df, title=industry, x="Date", y="Rolling mean" if roll_data[0] else "Close",
              color="Symbol", line_group="Symbol", hover_name="Symbol", line_shape="spline", render_mode="svg")
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [239]:
data_dict = {"Symbol": [], "Initial close": [], "Final close": []}
stocks = industries[industry]
for stock in stocks:
    data_dict["Symbol"].append(stock)
    temp_df = stock_df.loc[stock_df["Symbol"] == stock]
    print(temp_df.size-1)
    data_dict["Initial close"].append(temp_df.iloc[0]["Close"])
    data_dict["Final close"].append(temp_df.iloc[-1]["Close"])
data_df = pd.DataFrame.from_dict(data_dict)
data_df["Diff"] = data_df["Final close"] - data_df["Initial close"]
data_df = data_df.sort_values(by=["Diff"])
data_df

16367
36783


,Symbol,Initial close,Final close,Diff
0,MUNDRAPORT,962.9,135.50,-827.40
1,ADANIPORTS,140.0,730.05,590.05


## CandleStick Graph

In [240]:
import plotly.graph_objects as go
import pandas as pd
stock_name = industries[industry][0]
stock_df = all_stock_df.loc[all_stock_df["Symbol"] == stock_name]

In [241]:

fig = go.Figure(data=[go.Candlestick(x=stock_df['Date'],
                open=stock_df['Open'], high=stock_df['High'],
                low=stock_df['Low'], close=stock_df['Close']),
], layout=go.Layout(
    title=go.layout.Title(text=stock_name),
    shapes=[dict(
        x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper',
        line_width=2) for date in finance_minister_df.drop_duplicates('Prime minister', keep="first")["Date from"]],
    annotations=[dict(
        x=data[1]["Date from"], y=0.05, xref='x', yref='paper',
        showarrow=False, yanchor='auto', text=data[1]["Prime minister"]) for data in finance_minister_df.drop_duplicates('Prime minister', keep="first").iterrows()]
))
fig.show()


In [242]:
sector_df = all_stock_df.groupby(["Industry", "Date"]).mean()

sector_fig = go.Figure()
for industry in industries:
    data = sector_df.loc[industry].reset_index()
    data["Date"] = data["Date"].dt.strftime('%Y-%m-%d')
    sector_fig.add_trace(go.Scatter(x=data["Date"], y=data["Percent change"],
                                    mode='lines',
                                    name=industry))
sector_fig.show()

In [248]:
date_wise_mean_df = all_stock_df.groupby(["Date"]).mean().reset_index()
perc_change_start = date_wise_mean_df.iloc[0]["Percent change"]
perc_change_end = date_wise_mean_df.iloc[-1]["Percent change"]
gauge_value = 3 if perc_change_end > perc_change_start else 1
date_wise_mean_df

,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Trades,Percent change
0,2000-01-03,971.295455,1037.515152,1048.381818,1031.981818,1046.369697,1046.295455,1043.684848,NaN,2.156611
1,2000-01-04,1046.295455,1101.595455,1113.639394,1061.278788,1097.945455,1096.909091,1087.294848,NaN,-1.009929
2,2000-01-05,1096.909091,1048.757576,1095.516667,1017.303030,1048.309091,1047.637879,1061.890606,NaN,1.251855
3,2000-01-06,1047.637879,1053.346970,1072.887879,1000.918182,1019.696970,1019.443939,1028.510909,NaN,0.569018
4,2000-01-07,1019.443939,980.981818,994.213636,956.634848,971.571212,971.904545,972.593939,NaN,-0.575351
...,...,...,...,...,...,...,...,...,...,...
5301,2021-04-26,2520.724490,2536.018367,2566.816327,2512.578571,2538.279592,2537.928571,2539.116122,113355.469388,0.115421
5302,2021-04-27,2537.928571,2543.624490,2570.713265,2525.588776,2552.784694,2551.491837,2547.185714,117924.285714,0.944895
5303,2021-04-28,2551.491837,2561.095918,2612.457143,2544.588776,2585.656122,2588.959184,2583.668163,124308.857143,0.711136
5304,2021-04-29,2588.959184,2603.717347,2641.598980,2565.839796,2600.215306,2599.134694,2601.644694,138005.244898,-0.372824


In [271]:
trade_sum_df = all_stock_df.groupby(["Industry","Finance minister"]).sum()
list(trade_sum_df.loc["AUTOMOBILE"].Trades)

[308446035.0,
 0.0,
 5848999.0,
 292966857.0,
 94476895.0,
 33004029.0,
 62368726.0,
 0.0]